In [10]:
!pip install -U -q -r ../requirements.txt

In [20]:
import os, sys
from dotenv import load_dotenv

In [22]:
rpath = os.path.abspath('/home/user/Documents/10/w7/PromptBuilder for RAG/backend')

if rpath not in sys.path:
    sys.path.insert(0, rpath)

In [24]:
from service.file_processor import FileProcessor
from service.chroma_db_manager import ChromaDBManager
from service.rag_processor import RAGProcessor

In [25]:
from dotenv import load_dotenv
load_dotenv()

True

In [26]:
from langchain_openai import ChatOpenAI

In [30]:
llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"),model_name="gpt-3.5-turbo", temperature=0)

In [32]:
filePath = os.getenv("PATH_TO_TXT")

# Load and split the pdf file

In [35]:
file = FileProcessor(filePath)
file.load_file()
sp = file.split_file()
sp[30]

Document(page_content='Secondhand cars left us hitchhiking on the side of the freeway. A secondhand car was also the reason my mom got married. If it hadn’t been for the Volkswagen that didn’t work, we never would have looked for the mechanic who became the husband who became the stepfather who became the man who tortured us for years and put a bullet in the back of my mother’s head—I’ll take the new car with the warranty every time.', metadata={'source': '../data/data.txt'})

# Create and store it to chroma_db

In [38]:
chroma = ChromaDBManager()
chromaDb= chroma.store_and_load_chroma_db(sp)
chromaDb

In [39]:
chroma.search_chroma_db_with_score("grandmother")

[(Document(page_content='My great-grandmother lived with us as well. We called her Koko. She was super old, well into her nineties, stooped and frail, completely blind. Her eyes had gone white, clouded over by cataracts. She couldn’t walk without someone holding her up. She’d sit in the kitchen next to the coal stove, bundled up in long skirts and head scarves, blankets over her', metadata={'source': '../data/data.txt'}),
  0.2524379789829254),
 (Document(page_content='My grandmother Frances Noah was the family matriarch. She ran the house, looked after the kids, did the cooking and the cleaning. She’s barely five feet tall, hunched over from years in the factory, but rock hard and still to this day very active and very much alive. Where my grandfather was big and boisterous, my grandmother was calm, calculating, with a mind as sharp as anything. If you need to know anything in the family history, going back to the 1930s, she can tell you what day it happened, where it happened, and wh

# RAG

In [42]:
rag_processor = RAGProcessor(llm)
rag_processor.create_rag(chromaDb)
      

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7aedcacd2dd0>, search_type='similarity_score_threshold', search_kwargs={'score_threshold': 0.5})

In [44]:
# create the chain with the templet 
chain = rag_processor.process_rag_chain() 
chain

{
  context: RunnableLambda(itemgetter('question'))
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7aedcacd2dd0>, search_type='similarity_score_threshold', search_kwargs={'score_threshold': 0.5}),
  question: RunnableLambda(itemgetter('question'))
}
| RunnableAssign(mapper={
    context: RunnableLambda(itemgetter('context'))
  })
| {
    response: ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="\n            You are an expert LLM prompt writing service.\n            You take their prompt as input and output a better prompt based on your prompt writing expertise and the knowledge on the context. \n            You must write 5 top prompts that can achieve their desired objective and expected outputs, please respond with 'I don't know':\n            {question}\n            Context:\n 

In [48]:
prompt = "tell me about Koko"

reault= rag_processor.invoker(prompt)
reault["response"].content

"1. Explore the unique relationship between Koko and the narrator, delving into the dynamics of communication and companionship despite physical limitations.\n2. Reflect on the significance of Koko's presence in the household, considering her role as a silent yet integral member of the family.\n3. Discuss the impact of Koko's immobility on the narrator's perception of her, highlighting the complexities of human connection beyond physical interaction.\n4. Examine the daily routines and interactions involving Koko, shedding light on the moments of tenderness and routine that define their shared experiences.\n5. Analyze the symbolism of Koko's constant presence by the coal stove, drawing parallels between her warmth and the emotional comfort she provides to those around her."

# EVALUATION 

In [58]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [52]:
file = FileProcessor(filePath)
file.load_file()
documents = file.split_file()


In [63]:
generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-4")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)


testset = generator.generate_with_langchain_docs(documents, test_size=3, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

embedding nodes:   0%|          | 0/1974 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/10 [00:00<?, ?it/s]

Exception in thread Thread-12:
Traceback (most recent call last):
  File "/home/user/anaconda3/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/home/user/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 95, in run
    results = self.loop.run_until_complete(self._aresults())
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/anaconda3/lib/python3.11/asyncio/base_events.py", line 653, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/home/user/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 83, in _aresults
    raise e
  File "/home/user/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 78, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "/home/user/anaconda3/lib/python3.11/asyncio/tasks.py", line 615, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "/home/user/anaconda3/lib/python3.1

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

In [66]:
generator = TestsetGenerator.with_openai()

testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

/tmp/ipykernel_116957/1616921231.py:1: DeprecationWarning: The function with_openai was deprecated in 0.1.4, and will be removed in the 0.2.0 release. Use from_langchain instead.
  generator = TestsetGenerator.with_openai()


embedding nodes:   0%|          | 0/1974 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/10 [00:00<?, ?it/s]

Exception in thread Thread-14:
Traceback (most recent call last):
  File "/home/user/anaconda3/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/home/user/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 95, in run
    results = self.loop.run_until_complete(self._aresults())
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/anaconda3/lib/python3.11/asyncio/base_events.py", line 653, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/home/user/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 83, in _aresults
    raise e
  File "/home/user/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 78, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "/home/user/anaconda3/lib/python3.11/asyncio/tasks.py", line 615, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "/home/user/anaconda3/lib/python3.1

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

In [ ]:
testset

Task exception was never retrieved
future: <Task finished name='Task-18056' coro=<AsyncClient.aclose() done, defined at /home/user/anaconda3/lib/python3.11/site-packages/httpx/_client.py:1996> exception=RuntimeError('Event loop is closed')>
Traceback (most recent call last):
  File "/home/user/anaconda3/lib/python3.11/site-packages/httpx/_client.py", line 2003, in aclose
    await self._transport.aclose()
  File "/home/user/anaconda3/lib/python3.11/site-packages/httpx/_transports/default.py", line 383, in aclose
    await self._pool.aclose()
  File "/home/user/anaconda3/lib/python3.11/site-packages/httpcore/_async/connection_pool.py", line 324, in aclose
    await connection.aclose()
  File "/home/user/anaconda3/lib/python3.11/site-packages/httpcore/_async/connection.py", line 173, in aclose
    await self._connection.aclose()
  File "/home/user/anaconda3/lib/python3.11/site-packages/httpcore/_async/http11.py", line 253, in aclose
    await self._network_stream.aclose()
  File "/home/u

In [ ]:
test_df = testset.to_pandas()

test_df



In [5]:
answers = []
contexts = []

for question in test_questions:
  response =rag_processor.invoker(prompt)
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])


NameError: name 'test_questions' is not defined

In [8]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})
response_dataset[0]

NameError: name 'test_questions' is not defined

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

results = evaluate(response_dataset, metrics)

results



In [ ]:
results_df = results.to_pandas()
results_df